In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pvlib
from pvlib.pvsystem import PVSystem
from pvlib.location import Location
from pvlib.modelchain import ModelChain

In [3]:
# Importing the plant configuration from the site_config.csv
site_config = pd.read_csv('../site_config.csv')

interest = 1
site_config = site_config.loc[interest]
site_config

Plant_name                                                Narnaul
Latitude                                                  28.1002
Longitude                                                   75.99
Timezone                                            Asia/Calcutta
Elev                                                          315
surface_tilt                                                   40
surface_azimuth                                                 0
albedo                                                        0.2
series_modules                                                 20
parallel_modules                                              172
module_name                     Vikram_Solar_Eldora_VSP_72_315_03
module_count                                                 3440
module_wattage                                                315
inverter_name       ABB__MICRO_0_25_I_OUTD_US_208_208V__CEC_2014_
inverter_count                                                  1
inv_pmax  

In [4]:
# load some module and inverter specifications
cec_modules = pvlib.pvsystem.retrieve_sam('CECMod')
sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')
cec_inverters = pvlib.pvsystem.retrieve_sam('CECInverter')
cec_module = cec_modules[site_config.module_name]
cec_inverter = cec_inverters[site_config.inverter_name]
sandia_module = sandia_modules['Canadian_Solar_CS5P_220M___2009_']

In [5]:
location = Location(site_config.Latitude,site_config.Longitude,
							site_config.Timezone,site_config.Elev, site_config.Plant_name)
# location = Location(latitude=26, longitude=76)
location

Location: 
  name: Narnaul
  latitude: 28.100240000000003
  longitude: 75.99
  altitude: 315
  tz: Asia/Calcutta

In [6]:
system = PVSystem(surface_tilt=site_config.surface_tilt,
                  surface_azimuth=site_config.surface_azimuth,
                  albedo=0.25,
                  module=site_config.module_name,
                  module_parameters=cec_module,
                  modules_per_string=site_config.series_modules,
                  strings_per_inverter=site_config.parallel_modules,
                  inverter='ABB PVS800',
                  inverter_parameters=cec_inverter,
                  racking_model='open_rack_cell_glassback',
                  model='cec', name='Narnaul')
# system.module_parameters = system.module_parameters.append(pd.Series([1, -0.002438, 0.00031, -1.246E-05, 2.11E-07, -1.36E-09], index=['B0','B1','B2','B3','B4','B5']))
# system.module_parameters

In [7]:
mc = ModelChain(system, location, orientation_strategy='south_at_custom',dc_model='cec', clearsky_model='ineichen', transposition_model='haydavies',
                solar_position_method='nrel_numpy', airmass_model='kastenyoung1989',ac_model=None, aoi_model='no_loss',spectral_model='no_loss',
                temp_model='sapm',losses_model='no_loss', name=location.name+'_Modelchain')
mc

ModelChain: 
  name: Narnaul_Modelchain
  orientation_strategy: south_at_custom
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: cec
  ac_model: snlinverter
  aoi_model: no_aoi_loss
  spectral_model: no_spectral_loss
  temp_model: sapm_temp
  losses_model: no_extra_losses

In [8]:
# Importing the weather data
weather = pd.read_csv('../Narnaul_site/data/weather_data/selected_data.csv', index_col='Timestamp')

In [9]:
weather.index = pd.to_datetime(weather.index.values)
weather.index = weather.index.tz_localize('Asia/Kolkata')
mc.run_model(times=weather.index, weather=weather)

ModelChain: 
  name: Narnaul_Modelchain
  orientation_strategy: south_at_custom
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: cec
  ac_model: snlinverter
  aoi_model: no_aoi_loss
  spectral_model: no_spectral_loss
  temp_model: sapm_temp
  losses_model: no_extra_losses

In [10]:
plt.figure(figsize=[20,5])
plt.plot(mc.dc.p_mp.values)

In [13]:
mc.dc

i_sc        v_oc         i_mp        v_mp  \
2017-12-18 08:30:00+05:30   577.031067  855.677335   535.224255  724.157125   
2017-12-18 08:35:00+05:30   617.660634  855.203888   572.729579  722.599944   
2017-12-18 08:40:00+05:30   657.248568  854.593451   609.258045  720.928478   
2017-12-18 08:50:00+05:30   733.420527  853.068271   679.516790  717.334614   
2017-12-18 09:05:00+05:30   840.572914  850.292970   778.170140  711.719069   
2017-12-18 09:20:00+05:30   939.689294  847.215308   869.141590  706.143493   
2017-12-18 09:25:00+05:30   971.007605  846.159000   898.082679  704.118799   
2017-12-18 09:30:00+05:30  1001.480777  845.097033   925.697573  702.528787   
2017-12-18 09:35:00+05:30  1031.114856  844.034056   953.128719  700.503735   
2017-12-18 09:40:00+05:30  1059.914191  842.974173   979.215979  698.919549   
2017-12-18 09:45:00+05:30  1087.881672  841.921030  1004.541497  697.342719   
2017-12-18 09:50:00+05:30  1115.018901  840.877893  1029.758289  695.341581   
2017-12-18 09:55:00+05:30  1141.326391  839.847698  1053.849002  693.622062   
2017-12-18 10:00:00+05:30  1166.803700  838.833105  1076.948554  692.083042   
2017-12-18 10:05:00+05:30  1191.449591  837.836533  1099.305898  690.560572   
2017-12-18 10:10:00+05:30  1215.262152  836.860189  1120.923962  689.056423   
2017-12-18 10:15:00+05:30  1238.238895  835.906098  1141.804985  687.572210   
2017-12-18 10:20:00+05:30  1260.376881  834.976121  1162.230791  685.944665   
2017-12-18 10:25:00+05:30  1281.672782  834.071974  1181.646935  684.504798   
2017-12-18 10:30:00+05:30  1302.122985  833.195241  1200.329005  683.088917   
2017-12-18 10:35:00+05:30  1321.723641  832.347389  1218.277132  681.698155   
2017-12-18 10:40:00+05:30  1340.470748  831.529779  1235.009272  680.599010   
2017-12-18 10:50:00+05:30  1375.387796  829.990232  1266.726182  678.216497   
2017-12-18 11:00:00+05:30  1406.840833  828.585631  1295.477897  675.948236   
2017-12-18 11:05:00+05:30  1421.258032  827.936405  1308.166339  675.154297   
2017-12-18 11:10:00+05:30  1434.797005  827.323734  1320.420544  674.227341   
2017-12-18 11:20:00+05:30  1459.225022  826.211173  1343.209472  672.203361   
2017-12-18 11:25:00+05:30  1470.106842  825.712684  1352.891455  671.534193   
2017-12-18 11:30:00+05:30  1480.096037  825.253558  1361.280491  671.160796   
2017-12-18 11:40:00+05:30  1497.384366  824.455557  1377.526741  669.660073   
...                                ...         ...          ...         ...   
2018-05-10 11:00:00+05:30  1273.830391  834.406000  1174.198489  685.209928   
2018-05-10 11:05:00+05:30  1286.262532  833.875917  1185.568803  684.343898   
2018-05-10 11:15:00+05:30  1308.904632  832.902714  1206.112998  682.849091   
2018-05-10 11:20:00+05:30  1319.099512  832.461318  1215.566644  682.057217   
2018-05-10 11:25:00+05:30  1328.534440  832.051097  1224.077139  681.455490   
2018-05-10 11:30:00+05:30  1337.203191  831.672746  1232.230125  680.716027   
2018-05-10 11:35:00+05:30  1345.100069  831.326900  1238.943689  680.432955   
2018-05-10 11:40:00+05:30  1352.219901  831.014129  1245.779609  679.747705   
2018-05-10 11:45:00+05:30  1358.558033  830.734944  1251.650824  679.254136   
2018-05-10 11:50:00+05:30  1364.110340  830.489797  1256.363309  679.053691   
2018-05-10 11:55:00+05:30  1368.873215  830.279079  1261.200908  678.452525   
2018-05-10 12:00:00+05:30  1372.843573  830.103125  1264.569148  678.308747   
2018-05-10 12:05:00+05:30  1376.018850  829.962213  1267.261036  678.193602   
2018-05-10 12:10:00+05:30  1378.397003  829.856564  1269.276071  678.107272   
2018-05-10 12:15:00+05:30  1379.976506  829.786341  1270.613890  678.049891   
2018-05-10 12:20:00+05:30  1380.756350  829.751655  1271.274257  678.021547   
2018-05-10 12:25:00+05:30  1380.736044  829.752559  1271.257063  678.022285   
2018-05-10 12:30:00+05:30  1379.915610  829.789050  1270.562319  678.052104   
2018-05-10 12:35:00+05:30  1378.295587  829.861071  1269.190159  678.110955   
2018-